<h1><center>Extracting and Visualizing Stock Data The Top 5 Companies by Market Cap</h1>
<h2>Description</h2>


Extracting essential data from a dataset and displaying it is a necessary part of data science; therefore individuals can make correct decisions based on the data.The data extraction will be done using Python's BeautifulSoup and Yahoo Finance. 


<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Define a Function that Makes a Graph</li>
        <li> Use yfinance to Extract Tesla Stock Data</li>
        <li>Use Webscraping to Extract Tesla Revenue Data</li>
        <li> Use yfinance to Extract Stock Data from remaining companies</li>
        <li> Use Webscraping to Revenue Data from remaining companies</li>
        <li>Plot Stock Graphs</li>
    </ul>
<p>
</div>

<hr>


In [31]:
!pip install yfinance
#!pip install pandas
#!pip install requests
!pip install bs4
#!pip install plotly

In [32]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define Graphing Function


We define and make use of the function `make_graph`. It takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.


In [33]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

The following code will use Tesla as an example.

## Use yfinance to Extract Stock Data


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is `TSLA`.


In [34]:
tesla = yf.Ticker("TSLA")

Using the ticker object and the function `history`, we extract stock information and save it in a dataframe named `tesla_data`. The `period` parameter is set to `max` so we get information for the maximum amount of time.


In [35]:
tesla_data=tesla.history(period="max")

We will **Reset the index** using the `reset_index(inplace=True)` function on the tesla_data DataFrame and display the first five rows of the `tesla_data` dataframe using the `head` function. 


In [36]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,3.800,5.000,3.508,4.778,93831500,0,0.0
1,2010-06-30,5.158,6.084,4.660,4.766,85935500,0,0.0
2,2010-07-01,5.000,5.184,4.054,4.392,41094000,0,0.0
3,2010-07-02,4.600,4.620,3.742,3.840,25699000,0,0.0
4,2010-07-06,4.000,4.000,3.166,3.222,34334500,0,0.0


##  Use Webscraping to Extract Tesla Revenue Data


We use the `requests` library to download the webpage [https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue](https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0220ENSkillsNetwork23455606-2021-01-01). 

In [37]:
url =" https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url).text

Parse the html data using `beautiful_soup`.


In [38]:
soup = BeautifulSoup(html_data, "html5lib")

Next we extract the table with `Tesla Quarterly Revenue` and store it into a dataframe named `tesla_revenue`. The dataframe has columns `Date` and `Revenue`.


In [39]:
# Extract all tables
tables = soup.find_all('table')

# Look for table with "Tesla Quarterly Revenue"
for index, table in enumerate(tables):
    if ("Tesla Quarterly Revenue" in str(table)):
        table_index = index
print(table_index)        

1


In [40]:
tesla_revenue = pd.DataFrame(columns=["Date","Revenue"])

#Append the column entries to our DataFrame
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        date = col[0].text
        revenue = col[1].text
        tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

tesla_revenue.head()

,Date,Revenue
0,2021-06-30,"$11,958"
1,2021-03-31,"$10,389"
2,2020-12-31,"$10,744"
3,2020-09-30,"$8,771"
4,2020-06-30,"$6,036"


Execute the following line to remove the comma and dollar sign from the `Revenue` column.


In [41]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

<ipython-input-41-2aef5327de36>:1: FutureWarning:

The default value of regex will change from True to False in a future version.



Execute the following lines to remove an null or empty strings in the Revenue column.


In [42]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [43]:
tesla_revenue.tail()

,Date,Revenue
43,2010-09-30,31
44,2010-06-30,28
45,2010-03-31,21
47,2009-09-30,46
48,2009-06-30,27


##  Stock Data for Apple, Microsoft, Alphabet, Amazon and Facebook


We will now repeat the above process to get the data for Apple, Microsoft, Alphabet, Amazon and Facebook.

### Apple


In [44]:
#Use the Ticker function to extract data on the stock, to create a ticker object
apple= yf.Ticker("AAPL")
aapl_data=apple.history(period="max")

aapl_data.reset_index(inplace=True)

# Download, store and parse the data
url = "https://www.macrotrends.net/stocks/charts/AAPL/apple/revenue"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")

# Locate the table we want
tables = soup.find_all("table")
for index, table in enumerate(tables):
    if ("Apple Quarterly Revenue" in str(tables)):
        table_index = index

# Create a DataFrame with Date and Revenue columns      
aapl_revenue = pd.DataFrame(columns= ["Date", "Revenue"])   

# Append dates and revenue data to DataFrame
for row in tables[1].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        date = col[0].text
        revenue = col[1].text
        aapl_revenue = aapl_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

# Remove commas, $ and white spaces
aapl_revenue["Revenue"] = aapl_revenue['Revenue'].str.replace(',|\$',"")
aapl_revenue.dropna(inplace=True)
aapl_revenue = aapl_revenue[aapl_revenue['Revenue'] != ""]
       
aapl_revenue.tail()    

<ipython-input-44-f967a0fec27b>:30: FutureWarning:

The default value of regex will change from True to False in a future version.



,Date,Revenue
61,2006-03-31,4359
62,2005-12-31,5749
63,2005-09-30,3678
64,2005-06-30,3520
65,2005-03-31,3243


### Microsoft

In [45]:
#Use the Ticker function to extract data on the stock, to create a ticker object
microsoft= yf.Ticker("MSFT")
msft_data=microsoft.history(period="max")

msft_data.reset_index(inplace=True)

# Download, store and parse the data
url = "https://www.macrotrends.net/stocks/charts/MSFT/microsoft/revenue"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")

# Locate the table we want
tables = soup.find_all("table")
for index, table in enumerate(tables):
    if ("Microsoft Quarterly Revenue" in str(tables)):
        table_index = index

# Create a DataFrame with Date and Revenue columns      
msft_revenue = pd.DataFrame(columns= ["Date", "Revenue"])   

# Append dates and revenue data to DataFrame
for row in tables[1].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        date = col[0].text
        revenue = col[1].text
        msft_revenue = msft_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

# Remove commas, $ and white spaces
msft_revenue["Revenue"] = msft_revenue['Revenue'].str.replace(',|\$',"")
msft_revenue.dropna(inplace=True)
msft_revenue = msft_revenue[msft_revenue['Revenue'] != ""]
       
msft_revenue.tail()    

<ipython-input-45-d41c7847e509>:30: FutureWarning:

The default value of regex will change from True to False in a future version.



,Date,Revenue
61,2006-03-31,10900
62,2005-12-31,11837
63,2005-09-30,9741
64,2005-06-30,10161
65,2005-03-31,9620


### Alphabet

In [46]:
#Use the Ticker function to extract data on the stock, to create a ticker object
alphabet= yf.Ticker("GOOGL")
googl_data=alphabet.history(period="max")

googl_data.reset_index(inplace=True)

# Download, store and parse the data
url = "https://www.macrotrends.net/stocks/charts/GOOGL/alphabet/revenue"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")

# Locate the table we want
tables = soup.find_all("table")
for index, table in enumerate(tables):
    if ("Alphabet Quarterly Revenue" in str(tables)):
        table_index = index

# Create a DataFrame with Date and Revenue columns      
googl_revenue = pd.DataFrame(columns= ["Date", "Revenue"])   

# Append dates and revenue data to DataFrame
for row in tables[1].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        date = col[0].text
        revenue = col[1].text
        googl_revenue = googl_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

# Remove commas, $ and white spaces
googl_revenue["Revenue"] =googl_revenue['Revenue'].str.replace(',|\$',"")
googl_revenue.dropna(inplace=True)
googl_revenue = googl_revenue[googl_revenue['Revenue'] != ""]
       
googl_revenue.tail()    

<ipython-input-46-9074f2d40afb>:30: FutureWarning:

The default value of regex will change from True to False in a future version.



,Date,Revenue
61,2006-03-31,2254
62,2005-12-31,1919
63,2005-09-30,1578
64,2005-06-30,1384
65,2005-03-31,1257


### Amazon

In [51]:
#Use the Ticker function to extract data on the stock, to create a ticker object
amazon= yf.Ticker("AMZN")
amzn_data=amazon.history(period="max")

amzn_data.reset_index(inplace=True)

# Download, store and parse the data
url = "https://www.macrotrends.net/stocks/charts/AMZN/amazon/revenue"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html5lib")

# Locate the table we want
tables = soup.find_all("table")
for index, table in enumerate(tables):
    if ("Amazon Quarterly Revenue" in str(tables)):
        table_index = index

# Create a DataFrame with Date and Revenue columns      
amzn_revenue = pd.DataFrame(columns= ["Date", "Revenue"])   

# Append dates and revenue data to DataFrame
for row in tables[1].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        date = col[0].text
        revenue = col[1].text
        amzn_revenue = amzn_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

# Remove commas, $ and white spaces
amzn_revenue["Revenue"] = amzn_revenue['Revenue'].str.replace(',|\$',"")
amzn_revenue.dropna(inplace=True)
amzn_revenue = amzn_revenue[amzn_revenue['Revenue'] != ""]
       
amzn_revenue.tail()    

<ipython-input-51-c569ff048ebf>:30: FutureWarning:

The default value of regex will change from True to False in a future version.



,Date,Revenue
61,2006-03-31,2279
62,2005-12-31,2977
63,2005-09-30,1858
64,2005-06-30,1753
65,2005-03-31,1902


## Plot Stock Graphs


We make use of the `make_graph` function  we defined earlier to graph the Stock Data. The structure to call the `make_graph` function is `make_graph(name_data, name_revenue, 'Name')`. 


### Tesla

In [57]:
make_graph(tesla_data,tesla_revenue,'Tesla')

### Apple

In [53]:
make_graph(aapl_data,aapl_revenue,'Apple')

### Microsoft

In [54]:
make_graph(msft_data,msft_revenue,'Microsoft')

### Alphabet

In [55]:
make_graph(googl_data,googl_revenue,'Alphabet')

In [56]:
make_graph(amzn_data,amzn_revenue,'Amazon')

<h2>Credits</h2> 

<p>This project was inspired by:</p>

<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0220ENSkillsNetwork23455606-2021-01-01">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

Azim Hirjani


<h2>Author</h2> 

<p>Mudiwa Matanda is currently a final year physics student at the University of Pretoria.</p>
